In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import seaborn as sns
sns.set()

In [2]:
raw_data = pd.read_csv("C:/Users/stunn/Desktop/Assessment - Snr-Principal Data Analyst_Jan 2024 Final External v2/SectionA_Job Title with Labelled SSOC.csv")
raw_data.head()

,MCF_ID,Job Title,Labelled SSOC,Labelled SSOC Title
0,TJM97987719591,Admin/Receptionist,42241,Receptionist (general)
1,TJM97987819310,Beautician Supervisor,51421,Beautician
2,TJM97987859830,Senior / Quantity Surveyor (C&S/Tender/Project),21494,Quantity surveyor
3,TJM97987826405,Building and Construction Site Engineer,21422,Building construction engineer
4,TJM97987876381,Senior Software Engineer,25121,Software developer


# Data cleaning

## Basic info

In [5]:
raw_data.shape

(2359, 4)

In [6]:
raw_data.isnull().sum()

MCF_ID                 0
Job Title              0
Labelled SSOC          0
Labelled SSOC Title    0
dtype: int64

In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2359 entries, 0 to 2358
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   MCF_ID               2359 non-null   object
 1   Job Title            2359 non-null   object
 2   Labelled SSOC        2359 non-null   int64 
 3   Labelled SSOC Title  2359 non-null   object
dtypes: int64(1), object(3)
memory usage: 73.8+ KB


In [8]:
raw_data.nunique()

MCF_ID                 2359
Job Title              1938
Labelled SSOC           363
Labelled SSOC Title     363
dtype: int64

In [9]:
raw_data.duplicated().sum()

0

## Change datatype

In [11]:
raw_data['Labelled SSOC Title'] = raw_data['Labelled SSOC Title'].astype('category')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2359 entries, 0 to 2358
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   MCF_ID               2359 non-null   object  
 1   Job Title            2359 non-null   object  
 2   Labelled SSOC        2359 non-null   int64   
 3   Labelled SSOC Title  2359 non-null   category
dtypes: category(1), int64(1), object(2)
memory usage: 71.0+ KB


## Check point

In [13]:
df = raw_data.copy()

## Slice table for analysis

In [15]:
new_table = df[['Labelled SSOC','Labelled SSOC Title']].copy()
new_table = new_table.drop_duplicates()

# Analyze

In [17]:
df.head()

,MCF_ID,Job Title,Labelled SSOC,Labelled SSOC Title
0,TJM97987719591,Admin/Receptionist,42241,Receptionist (general)
1,TJM97987819310,Beautician Supervisor,51421,Beautician
2,TJM97987859830,Senior / Quantity Surveyor (C&S/Tender/Project),21494,Quantity surveyor
3,TJM97987826405,Building and Construction Site Engineer,21422,Building construction engineer
4,TJM97987876381,Senior Software Engineer,25121,Software developer


**Show counts of each labelled SSOC title**

In [19]:
#group categories by number of times Labelled SSOC title appeared

grouped = df.groupby(['Labelled SSOC Title'], observed=True)['Labelled SSOC Title'].count().rename('Count') \
            .reset_index().sort_values('Count', ascending=True)
grouped.head()

,Labelled SSOC Title,Count
181,Library clerk,1
182,Library officer,1
187,Maintenance planner,1
188,Management consultant,1
194,Marine superintendent,1


In [20]:
#count number of times 1 appeared

count_ones = (grouped == 1).sum()
count_ones

Labelled SSOC Title      0
Count                  142
dtype: int64

In [21]:
#count number of times each count appeared

grouped_2 = grouped.groupby(['Count'], observed=True)['Count'].count().sort_values(ascending=False)
#pd.set_option('display.max_rows', None)  # Show all rows

grouped_2.head(5)

Count
1    142
2     55
3     30
4     20
6     15
Name: Count, dtype: int64

# Model

In [23]:
vectorizer = TfidfVectorizer()                             # Vectorization
X = vectorizer.fit_transform(df['Job Title'])

y = df['Labelled SSOC Title']                              #Dependent variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)    #Train-test split

clf = LogisticRegression()                                   #Train model
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)                               #Predict test data

#print("Classification Report:\n")                         #Evaluation
#print(classification_report(y_test, y_pred))

# Prediction (test)

In [25]:
df_pred = pd.DataFrame(y_pred, columns=["Prediction"])
df_pred.head()

,Prediction
0,Civil engineer
1,Welder and flame cutter
2,Business development manager
3,Software developer
4,Business consultant


In [26]:
df_y_test = pd.DataFrame(y_test, columns=["Labelled SSOC Title"])
df_y_test.head()

,Labelled SSOC Title
700,Building construction engineer
450,Welder and flame cutter
1992,Business development manager
1677,Software developer
2022,ICT sales and services professional


In [27]:
combined_df = pd.concat([df_y_test.reset_index(drop=True), df_pred.reset_index(drop=True)], axis=1)
combined_df.head(10)

,Labelled SSOC Title,Prediction
0,Building construction engineer,Civil engineer
1,Welder and flame cutter,Welder and flame cutter
2,Business development manager,Business development manager
3,Software developer,Software developer
4,ICT sales and services professional,Business consultant
5,Hair stylist/Hairdresser,Web and mobile applications developer
6,Cloud specialist,Cloud specialist
7,Cloud specialist,Cloud specialist
8,Software developer,Software developer
9,Web and mobile applications developer,Web and mobile applications developer


In [28]:
combined_df.shape

(472, 2)

**Count number of mismatches**

In [30]:
combined_df['match'] = combined_df['Prediction'] == combined_df['Labelled SSOC Title']

# Count matches and mismatches
num_matches = combined_df['match'].sum()
num_mismatches = len(combined_df) - num_matches

print(f"Number of matches: {num_matches}")
print(f"Number of mismatches: {num_mismatches}")

Number of matches: 288
Number of mismatches: 184


**Show rows with mismatches**

In [32]:
mismatches = combined_df[combined_df['Prediction'] != combined_df['Labelled SSOC Title']]
mismatches.head()

,Labelled SSOC Title,Prediction,match
0,Building construction engineer,Civil engineer,False
4,ICT sales and services professional,Business consultant,False
5,Hair stylist/Hairdresser,Web and mobile applications developer,False
12,Floor/Wall tiler,Web and mobile applications developer,False
17,Healthcare assistant (e.g. therapy aide/assist...,Warehouse worker,False


**Mismatched group counts > 10**

In [34]:
prediction_counts = combined_df.groupby('Prediction').size().reset_index(name='count')

filtered_predictions = prediction_counts[prediction_counts['count'] > 10]              #more than 10 in group
filtered_predictions = filtered_predictions.sort_values(by='count', ascending=False)   #sort by descending count

filtered_predictions

,Prediction,count
70,Web and mobile applications developer,67
66,Software developer,51
6,Business consultant,26
46,Marketing manager,25
51,Other car and light goods vehicle drivers,21
69,Warehouse worker,20
7,Business development executive,18
68,Waiter,15
57,Quantity surveyor,14


# Prediction (train)

In [36]:
X_train_pred = clf.predict(X_train)

In [37]:
df_pred_train = pd.DataFrame(X_train_pred, columns=["Prediction"])
df_pred_train.head()

,Prediction
0,Retail manager
1,Software developer
2,Marketing manager
3,Administration manager
4,IT support technician (including IT user helpd...


In [38]:
df_y_train = pd.DataFrame(y_train, columns=["Labelled SSOC Title"])
df_y_train.head()

,Labelled SSOC Title
1064,Retail manager
1377,Software developer
478,Occupational health and safety inspector
1911,Administration manager
744,IT support technician (including IT user helpd...


In [39]:
combined_df_train = pd.concat([df_y_train.reset_index(drop=True), df_pred_train.reset_index(drop=True)], axis=1)
combined_df_train.head()

,Labelled SSOC Title,Prediction
0,Retail manager,Retail manager
1,Software developer,Software developer
2,Occupational health and safety inspector,Marketing manager
3,Administration manager,Administration manager
4,IT support technician (including IT user helpd...,IT support technician (including IT user helpd...


In [40]:
combined_df_train.shape

(1887, 2)

In [41]:
combined_df_train['match'] = combined_df_train['Prediction'] == combined_df_train['Labelled SSOC Title']

# Count matches and mismatches
num_matches = combined_df_train['match'].sum()
num_mismatches = len(combined_df_train) - num_matches

print(f"Number of matches: {num_matches}")
print(f"Number of mismatches: {num_mismatches}")

Number of matches: 1417
Number of mismatches: 470


# Save as csv

combined_df.to_csv('predictions_vs_actual.csv', index=False)

## Prediction using user input

In [45]:
def predict_ssoc(job_title):                                #Inference
    job_vec = vectorizer.transform([job_title])
    predicted_ssoc = clf.predict(job_vec)[0]
    return predicted_ssoc

**User input**

In [47]:
user_input = 'data scientist'

In [48]:
print("Job title User input:", user_input)
print("Predicted Labelled SSOC:",predict_ssoc(user_input))      #predict labelled SSOC

Job title User input: data scientist
Predicted Labelled SSOC: Data scientist


**Show labelled SSOC**

In [50]:
result = new_table[new_table['Labelled SSOC Title'] == predict_ssoc(user_input)]
result

,Labelled SSOC,Labelled SSOC Title
93,21222,Data scientist
